In [ ]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
import pandas as pd
import yaml

# Params
dq = "influential_paper_detection_by_degreecentrality"
k = 3
top_k_papers_lt = pd.read_csv("../../data/query/cn/python/output_{dq}.csv".format(dq=dq)).head(k)["node"].tolist()
top_k_papers = "', '".join(x for x in top_k_papers_lt)


# DB credentials
with open("../../credentials.yaml", "r", encoding="utf-8") as f:
        credentials = yaml.safe_load(f)

database_group="CN_NEO4J"
keywords = ["sentiment analysis"]
uri = "{protocal}://{host}:{port}".format(protocal=credentials[database_group]["protocol"],
                                              host=credentials[database_group]["host"],
                                              port=credentials[database_group]["port"])

# Connect to Neo4j DB
neo4j_driver = GraphDatabase.driver(uri, auth=(credentials[database_group]["user"], credentials[database_group]["password"]))

condition = ""
for keyword in keywords:
        condition = condition + """
                                ((ANY(k in p1.keywords 
                                WHERE toLower(k) = "{keyword}")) 
                                OR (toLower(p1.abstract) CONTAINS "{keyword}")
                                OR (toLower(p1.title) CONTAINS "{keyword}"))
                                AND
                                ((ANY(k in p2.keywords 
                                WHERE toLower(k) = "{keyword}")) 
                                OR (toLower(p2.abstract) CONTAINS "{keyword}")
                                OR (toLower(p2.title) CONTAINS "{keyword}"))        
                        """.format(keyword=keyword)

cypher_query = """
                MATCH (p1:Paper)-[r]->(p2:Paper)
                WHERE
                {condition}
                RETURN p1, r, p2;
        """.format(condition=condition)

cypher_query = """
                MATCH (p1:Paper)-[:CITED*]->(p2:Paper)
                WHERE p2.nid in ['{top_k_papers}']
                RETURN *; 
                """.format(top_k_papers=top_k_papers)

cypher_query = """
                MATCH (p1:Paper)-[r:CITED]->(p2:Paper)
                WHERE 
                p1.cntCitation > 100
                AND
                p2.cntCitation > 100
                AND
                (p1.nid in ['{top_k_papers}'])
                AND 
                ((ANY(k in p2.keywords 
                        WHERE toLower(k) = "sentiment analysis")) 
                OR (toLower(p2.abstract) CONTAINS "sentiment analysis")
                OR (toLower(p2.title) CONTAINS "sentiment analysis"))
                OR
                (p2.nid in ['{top_k_papers}'])
                AND
                ((ANY(k in p1.keywords 
                        WHERE toLower(k) = "sentiment analysis")) 
                OR (toLower(p1.abstract) CONTAINS "sentiment analysis")
                OR (toLower(p1.title) CONTAINS "sentiment analysis"))
                RETURN *; 
                """.format(top_k_papers=top_k_papers)

# Load data as Graph (NetworkX) from Neo4j DB
res = neo4j_driver.session().run(cypher_query)

w = GraphWidget(graph=res.graph())

def custom_color_mapping(index, node):
    if node["properties"]["nid"] in top_k_papers_lt:
            color= "#E84A27"
    else:
            color="#45BB84"
    return color



w.set_node_color_mapping(custom_color_mapping)

w.show()

# Close the connection
neo4j_driver.close()
